# Simulation of the Process with Input Variations

Import PyRolL modules.

In [1]:
import pyroll.basic as pr
import pyroll.export as pre

Import the process data.

In [2]:
from weiner_variation.sim.process import PASS_SEQUENCE, IN_PROFILE, TEMPERATURE, DIAMETER, TEMPERATURE_STD, DIAMETER_STD, create_in_profile
from weiner_variation.sim.config import SAMPLE_COUNT, SEED
from weiner_variation.sim.data import DrawInput
from weiner_variation.config import DATA_DIR

Import further libraries.

In [3]:
import pandas as pd
from copy import deepcopy
from scipy.stats import norm
import numpy as np
import tqdm
from multiprocessing import Pool

Parameters for notebook using papermill.

In [4]:
DIAMETER_STD = DIAMETER_STD
TEMPERATURE_STD = TEMPERATURE_STD
OUTPUT_FILENAME = DATA_DIR / "sim_input_results.csv"

In [5]:
# Parameters
OUTPUT_FILENAME = "/home/maxwe/Git/pyroll/publication-weiner-variation/weiner_variation/data/sim_temperature_stds_results/0.1.csv"
TEMPERATURE_STD = 142.31500000000003


Create distribution functions for input values.

In [6]:
diameter_dist = norm(loc=DIAMETER, scale=DIAMETER_STD)
temperature_dist = norm(loc=TEMPERATURE, scale=TEMPERATURE_STD)

Draw random inputs from distributions.

In [7]:
RNG = np.random.default_rng(SEED)
diameters = diameter_dist.rvs(random_state=RNG, size=SAMPLE_COUNT)
temperatures = temperature_dist.rvs(random_state=RNG, size=SAMPLE_COUNT)

draws = [
    DrawInput(d, t)
    for d, t in zip(diameters, temperatures)
]

Define a worker function creating the in profile, running the solution procedure and extracting results.

In [8]:
def worker(draw: DrawInput):    
    ip = create_in_profile(draw.diameter)
    ip.temperature = draw.temperature

    sequence = deepcopy(PASS_SEQUENCE)
    sequence.solve(ip)       

    return pre.to_pandas(sequence)

Run the simulations using a process pool.

In [9]:
results = list(tqdm.tqdm(Pool().imap(worker, draws), total=SAMPLE_COUNT))

  0%|          | 0/10 [00:00<?, ?it/s]

 10%|█         | 1/10 [00:00<00:05,  1.59it/s]

 70%|███████   | 7/10 [00:01<00:00,  6.51it/s]

100%|██████████| 10/10 [00:01<00:00,  7.98it/s]

Create a dataframe from results.

In [10]:
df = pd.DataFrame([r.stack().swaplevel().sort_index() for r in results])
df

abs_draught                                                              \
           0         2         4         6         8         10        12   
0   -0.015344 -0.020258 -0.014426 -0.019869 -0.009266 -0.011217 -0.007984   
1   -0.015628 -0.020549 -0.014444 -0.019878 -0.009267 -0.011218 -0.007985   
2   -0.016036 -0.020961 -0.014468 -0.019890 -0.009270 -0.011219 -0.007985   
3   -0.015050 -0.019952 -0.014406 -0.019859 -0.009264 -0.011216 -0.007984   
4   -0.015639 -0.020560 -0.014444 -0.019878 -0.009267 -0.011218 -0.007985   
5   -0.015964 -0.020889 -0.014464 -0.019888 -0.009269 -0.011219 -0.007985   
6   -0.015202 -0.020111 -0.014416 -0.019864 -0.009265 -0.011217 -0.007984   
7   -0.015166 -0.020074 -0.014414 -0.019863 -0.009264 -0.011217 -0.007984   
8   -0.013705 -0.018395 -0.014291 -0.019800 -0.009253 -0.011211 -0.007983   
9   -0.014899 -0.019792 -0.014395 -0.019853 -0.009263 -0.011216 -0.007984   

                                 ... zouhar_contact_in_width            \
         14        16        18  ...                      8         10   
0 -0.010435 -0.004337 -0.005911  ...                   0.026  0.005702   
1 -0.010435 -0.004337 -0.005911  ...                   0.026  0.005701   
2 -0.010435 -0.004337 -0.005911  ...                   0.026  0.005701   
3 -0.010435 -0.004337 -0.005911  ...                   0.026  0.005702   
4 -0.010435 -0.004337 -0.005911  ...                   0.026  0.005701   
5 -0.010435 -0.004337 -0.005911  ...                   0.026  0.005701   
6 -0.010435 -0.004337 -0.005911  ...                   0.026  0.005702   
7 -0.010435 -0.004337 -0.005911  ...                   0.026  0.005702   
8 -0.010434 -0.004337 -0.005911  ...                   0.026  0.005704   
9 -0.010435 -0.004337 -0.005911  ...                   0.026  0.005702   

                                                                        
       12        14      16        18     20        22    24        26  
0  0.0198  0.003251  0.0148  0.004102  0.012  0.003359  0.01  0.002245  
1  0.0198  0.003251  0.0148  0.004102  0.012  0.003359  0.01  0.002245  
2  0.0198  0.003251  0.0148  0.004102  0.012  0.003359  0.01  0.002245  
3  0.0198  0.003252  0.0148  0.004102  0.012  0.003359  0.01  0.002245  
4  0.0198  0.003251  0.0148  0.004102  0.012  0.003359  0.01  0.002245  
5  0.0198  0.003251  0.0148  0.004102  0.012  0.003359  0.01  0.002245  
6  0.0198  0.003251  0.0148  0.004102  0.012  0.003359  0.01  0.002245  
7  0.0198  0.003251  0.0148  0.004102  0.012  0.003359  0.01  0.002245  
8  0.0198  0.003252  0.0148  0.004102  0.012  0.003359  0.01  0.002245  
9  0.0198  0.003252  0.0148  0.004102  0.012  0.003359  0.01  0.002245  

[10 rows x 3891 columns]

Save dataframe to CSV file.

In [11]:
df.to_csv(OUTPUT_FILENAME)